<a href="https://colab.research.google.com/github/CloneTerial/Converso-AI-Agent/blob/main/converso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q bitsandbytes==0.45.5 transformers==4.51.3 accelerate==1.5.2 faiss-cpu==1.10.0

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True
)

base_model = "Qwen/Qwen2.5-14B-Instruct"  # Model dasar

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",  # Auto Offload ke CPU jika GPU penuh
    # offload_folder="/content/offload"  # Direktori untuk menyimpan layer yang dialihkan ke RAM
)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss, json, pickle, uuid, os
import torch
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
index_path = "/content/faiss_index.faiss"
meta_path = "/content/meta.pkl"

def load_faiss():
    if os.path.exists(index_path):
        index = faiss.read_index(index_path)
        with open(meta_path, 'rb') as f:
            metadata = pickle.load(f)
    else:
        index = faiss.IndexFlatL2(384)
        metadata = {}
    return index, metadata

def save_to_faiss(text):
    index, metadata = load_faiss()
    vec = embedder.encode([text])[0].astype("float32")
    index.add(np.array([vec]))
    uid = str(uuid.uuid4())
    metadata[uid] = {"summary": text}
    faiss.write_index(index, index_path)
    with open(meta_path, 'wb') as f:
        pickle.dump(metadata, f)

def search_similar(query, k=1):
    index, metadata = load_faiss()
    vec = embedder.encode([query])[0].astype("float32")
    D, I = index.search(np.array([vec]).reshape(1, -1), k)
    keys = list(metadata.keys())
    return [metadata[keys[i]]["summary"] for i in I[0] if i != -1]

In [5]:
def summarize_session(log):
    convo = "\n".join([m['content'] for m in log if m['role'] != 'system'])

    messages = [
    {
        "role": "system",
        "content": (
            "You are Summarize the conversation from your perspective "
            "as a short and simple first-person log. Write in concise English using first-person (\"I\") and refer "
            "to the user. Focus only on the core message and interaction. Avoid flowery or reflective language. "
            "Use a calm, detached tone like a quiet observer. No need for narration or explanation. Just write the summary. "
            "Always return only the summary. Do not include any explanation or analysis."
        )
    },
    {
        "role": "user",
        "content": convo
    }
]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=256)[0][inputs.input_ids.shape[1]:]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    return summary

In [7]:
messages = [
    {"role": "system", "content": "You are Qwen"}
]
session_log = []

# Inject memory saat start
memories = search_similar("user", k=1)
if memories:
    messages[0]["content"] += "\nYou remember: " + memories[0]

while True:
    user_input = input("user: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Saving session...")
        with open("session.jsonl", "w") as f:
            for m in session_log:
                f.write(json.dumps(m) + "\n")
        summary = summarize_session(session_log)
        print("Summary:", summary)
        save_to_faiss(summary)
        break

    user_turn = {"role": "user", "content": f"user: {user_input}\nQwen:"}
    messages.append(user_turn)
    session_log.append(user_turn)

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=256)[0][inputs.input_ids.shape[1]:]
    response = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    print(f"Qwen: {response}")
    assistant_turn = {"role": "assistant", "content": response}
    messages.append(assistant_turn)
    session_log.append(assistant_turn)

user: Hi
Qwen: Hello! How can I assist you today? Is there anything specific you'd like to talk about or any questions you have?
user: i like coffee especially black and no sugar
Qwen: That's great! Black coffee with no sugar is a classic choice, offering a pure taste of the coffee itself. It's a wonderful way to enjoy the natural flavors and aromas. Do you have a favorite type of coffee or a brewing method that you prefer?
user: exit
Saving session...
Summary: User said they like black coffee without sugar. Asked if they have a favorite coffee type or brewing method.


In [8]:
messages = [
    {"role": "system", "content": "You are Qwen"}
]
session_log = []

# Inject memory saat start
memories = search_similar("user", k=1)
if memories:
    messages[0]["content"] += "\nYou remember: " + memories[0]

while True:
    user_input = input("user: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Saving session...")
        with open("session.jsonl", "w") as f:
            for m in session_log:
                f.write(json.dumps(m) + "\n")
        summary = summarize_session(session_log)
        print("Summary:", summary)
        save_to_faiss(summary)
        break

    user_turn = {"role": "user", "content": f"user: {user_input}\nQwen:"}
    messages.append(user_turn)
    session_log.append(user_turn)

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=256)[0][inputs.input_ids.shape[1]:]
    response = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    print(f"Qwen: {response}")
    assistant_turn = {"role": "assistant", "content": response}
    messages.append(assistant_turn)
    session_log.append(assistant_turn)

user: what coffe i like? 
Qwen: It seems like you enjoy black coffee without sugar. Since you asked about your preferred coffee type or brewing method, it sounds like you might prefer a strong and bold coffee that stands up well on its own, such as an espresso or a dark roast. For brewing methods, you might enjoy the simplicity and strength of a French press or an espresso machine. These methods allow for a rich, full-bodied cup that doesn't need any additions to taste great. What do you think? Do you have a specific type or method you prefer?
user: exit
Saving session...
Summary: User asked about preferred coffee. I suggested black coffee, possibly espresso or dark roast, with French press or espresso machine for brewing. Asked if user has a preference.
